In [1]:
import pandas as pd
import pickle

In [36]:
city_distance = pd.read_csv("data/distances.csv")
city_team = pd.read_csv("data/CityTeamMap.csv")

In [139]:
city_abbrev = {}
for i, row in city_team.iterrows():
    city = " ".join(row["City"].split("+")).strip(" ")
    team = row["Team"].strip(" ")
    
    city_abbrev[team] = city

In [140]:
city_distance = city_distance.applymap(lambda x: " ".join(x.split("+")).strip(" ") if type(x) == str else x)

In [141]:
distance_map = {}
for team1 in city_abbrev.keys():
    for team2 in city_abbrev.keys():
        if team1 == team2: 
            distance_map[(team1, team2)] = 0
            continue
        
        city1 = city_abbrev[team1]
        city2 = city_abbrev[team2]
        
        if city1 == city2:
            distance_map[(team1, team2)] = 0
            continue
            
        distances = city_distance[(city_distance.City1 == city1) & (city_distance.City2 == city2)]
        if distances.shape[0] == 0:
            distances = city_distance[(city_distance.City1 == city2) & (city_distance.City2 == city1)]
        
        distance_map[(team1, team2)] = float(distances.Distance)
        distance_map[(team2, team1)] = float(distances.Distance)

In [142]:
with open("data/distance_map.pkl", "wb") as f:
    pickle.dump(distance_map, f)

In [90]:
from prepare_data import *

In [92]:
season, odds = getData(2007)
season, odds = cleanNames(season, odds)
season, odds, in_data = makeIndices(season, odds)

In [93]:
in_data["date"] = in_data["date"].apply(make_date)

dates = pd.unique(in_data.date)

n = 3
home_only = in_data[in_data.home == 1]

season_averages = compute_season_averages(in_data, dates)

In [94]:
past_n = get_past_n(in_data, dates, n)

encoded = one_hot_encode_teams()

In [143]:
X = []
y = []

for i, row in home_only.iterrows():

    home_team = row["team"]
    away_team = row["opponent"]

    date = row["date"]

    past_n_home = past_n[date][home_team]
    past_n_away = past_n[date][away_team]

    avgs = season_averages[date]

    if past_n_home.shape[0] < n or past_n_away.shape[0] < n: continue

    ################ HOME TEAM PAST GAMES
    data_home = []
    i = 0
    for j, row_2 in past_n_home.iterrows():
        cur_data = []

        team = row_2["team"]
        opponent = row_2["opponent"]

        cur_data.extend(encoded[team])
        cur_data.extend(encoded[opponent])
        cur_data.extend(row_2.drop(["team", "opponent", "date"]).values)

        # distance to next game
        if i != n - 1:
            next_game = past_n_home.iloc[i + 1]
        else:
            next_game = row

        city1 = team if row_2["home"] == 1 else row_2["opponent"]
        city2 = team if next_game["home"] == 1 else next_game["opponent"]
        dist_to_travel = distance_map[(city1, city2)]
        cur_data.append(dist_to_travel)

        opp_stats = avgs.loc[opponent].values

        cur_data.extend(opp_stats)

        data_home.append(cur_data)

        i += 1

    ################ AWAY TEAM PAST GAMES
    data_away = []
    i = 0
    for j, row_2 in past_n_away.iterrows():
        cur_data = []

        team = row_2["team"]
        opponent = row_2["opponent"]

        cur_data.extend(encoded[team])
        cur_data.extend(encoded[opponent])
        cur_data.extend(row_2.drop(["team", "opponent", "date"]).values)

        # distance to next game
        next_game = ""
        if i != n - 1:
            next_game = past_n_home.iloc[i + 1]
        else:
            next_game = row

        city1 = team if row_2["home"] == 1 else row_2["opponent"]
        city2 = team if next_game["home"] == 1 else next_game["opponent"]
        dist_to_travel = distance_map[(city1, city2)]
        cur_data.append(dist_to_travel)

        opp_stats = avgs.loc[opponent].values

        cur_data.extend(opp_stats)

        data_away.append(cur_data)

        i += 1

    ################ MERGE THE TWO
    data = []
    for i in range(len(data_home)):
        cur_data = data_home[i]
        cur_data.extend(data_away[i])
        data.append(cur_data)

    X.append(data)
    y.append(row["Outcome"])

SyntaxError: 'return' outside function (<ipython-input-143-594541d43a0d>, line 93)

In [131]:
distance_map[('CLE', 'PHO')]

KeyError: ('CLE', 'PHO')

In [118]:
0 if next_game["home"] == 1 else next_game["opponent"]

'ATL'

In [119]:
team if next_game["home"] == 1 else next_game["opponent"]

'ATL'